In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import datetime as dt
pd.options.display.min_rows=1000
pd.set_option('display.max_rows', 1000)
pd.set_option('display.min_rows', 1000)
pd.set_option('display.expand_frame_repr', True)

import pypfopt
import matplotlib.pyplot as plt
import yfinance as yf
from pypfopt.expected_returns import mean_historical_return
from pypfopt import black_litterman, risk_models
from pypfopt.expected_returns import mean_historical_return
from pypfopt import EfficientFrontier
pd.options.display.float_format = '{:,.5f}'.format
from IPython.core.display import display, HTML
display(HTML("<style> .container{width:96% !important;}</style>"))
pd.options.display.max_rows=150
pd.options.display.max_columns=150

In [2]:
### load data
df_stock_return= pd.read_excel("Index Monthly Time Series.xlsx")
df_stock_return=df_stock_return.set_index("Date")
df_stock_return.head(3)

,Cash,BondGov,BondIG,BondHY,EquityUS,EquityEAFE,EquityEM
Date,,,,,,,
1988-01-31,0.00473,0.04684,0.03830,0.02909,0.03960,0.01674,0.09419
1988-02-29,0.00471,0.01222,0.01680,0.02756,0.03873,0.06558,0.00240
1988-03-31,0.00476,-0.02319,-0.01139,-0.00192,-0.03651,0.06036,0.09993


### Descriptive Statistics

In [3]:
#### Cumulative Return
(df_stock_return_data+1).cumprod().plot(figsize=(10,5));
plt.title("Cumulative Return")
plt.show()

NameError: name 'df_stock_return_data' is not defined

In [ ]:
def VaR_CVaR(data_series,
             significance_level):
    """Return VaR and CVaR and Tail Loss respectivley at the specifeid significance_level 
    1. VaR Based on Normal_Dist and sample mean and std 
    2. VaR Based on specified significance level quntile
    3. CVaR Based on avergae extreme values below VaR (VaR based on specified quantile)
    4. CVaR Based on Normal_Dist, sample mean and std, and VaR based on Quantile
    
    CVaR = (1/significance_level) * tail_loss
    """
    import scipy.stats
    import numpy as np
    mean=np.mean(data_series)
    std= np.std(data_series)
    #### Normal Dist VaR CVaR
    VaR_Normal_Dist=scipy.stats.norm.ppf(
        significance_level,
        loc = mean,
        scale = std)
    return_in_tail_based_on_VaR_Normal_Dist=data_series[data_series<VaR_Normal_Dist]
    
    tail_loss_Normal_dist = scipy.stats.norm.expect(lambda x: x,
                            loc = mean,
                            scale = std,
                            ub = VaR_Normal_Dist)
    CVaR_Normal = (1/significance_level) * tail_loss_Normal_dist
    
    
    #### T Dist Var CVaR
    t_dist_params = scipy.stats.t.fit(data_series)
    
    VaR_t_dist = scipy.stats.t.ppf(significance_level,*t_dist_params)
    tail_loss_t= scipy.stats.t.expect( lambda y: y, args = (t_dist_params[0],),
                                      loc = t_dist_params[1],
                                      scale = t_dist_params[2],
                                      ub = VaR_t_dist)
    
    CVaR_T = (1/significance_level) * tail_loss_t
    
    
    ### Data Driven VaR CVaR
    VaR_Quantile = np.quantile(data_series,significance_level)
    return_in_tail_based_on_VaR_Quantile=data_series[data_series<VaR_Quantile]
    CVaR_Quantile=np.mean(return_in_tail_based_on_VaR_Quantile)    
    
    
    #### VaR and CVaR GEV 
    genextreme_params= scipy.stats.genextreme.fit(-1*data_series)
    VaR_GEV = (scipy.stats.genextreme.ppf((1-significance_level), *genextreme_params))
    
    tail_loss_GEV = scipy.stats.genextreme.expect(lambda x: x, 
           args=(genextreme_params[0],),
                                                   loc = genextreme_params[1], 
                                              scale = genextreme_params[2],
                                              lb = VaR_GEV)
    CVaR_GEV = ((1/(significance_level)) * tail_loss_GEV)
    
    
    return ({"VaR_Normal_Dist":VaR_Normal_Dist,
             "VaR_GEV":VaR_GEV*-1,
             "VaR_t_dist":VaR_t_dist,
             "VaR_Quantile":VaR_Quantile,
             
            "CVaR_Normal":CVaR_Normal,
            "CVaR_GEV":CVaR_GEV*-1,
             "CVaR_T":CVaR_T,
             "CVaR_Quantile":CVaR_Quantile
            })

In [ ]:
def VaR_CVaR_Quantile(data_series,
                      significance_level):
    """Returnn Value at Risk (VaR) and Conditional Value at Risk (CVaR) at significance_level"""
    import scipy.stats
    import numpy as np
    mean=np.mean(data_series)
    std= np.std(data_series)
    ### Data Driven VaR CVaR
    VaR_Quantile = np.quantile(data_series,significance_level)
    return_in_tail_based_on_VaR_Quantile=data_series[data_series<VaR_Quantile]
    CVaR_Quantile=(np.mean(return_in_tail_based_on_VaR_Quantile))
    return(VaR_Quantile,CVaR_Quantile)

In [ ]:
def portfolio_return_series_computer(df_return_data,
                                     df_weights,
                                     list_of_assets_columns):
    """This function is to create return series of a portfolo taking weights of assest and return of assest"""
    df_portfolios_return=df_return_data[list_of_assets_columns].dot(
        df_weights[list_of_assets_columns].T)
    #df_portfolios_return.reset_index(inplace=True)
    df_portfolios_return=pd.melt(df_portfolios_return,var_name="portfolio",value_name="return")
    return(df_portfolios_return)

In [ ]:
results.summary()

In [ ]:
def portfolio_stats_computer(portfolio_return_series,
                            frequency=252):
    """This Function is to computer Annualized Return and Volatility of a series given the series and its frequency
    VaR and CVaR are at 5%"""
    portfolio_return_series=pd.Series(portfolio_return_series)
    
    portfolio_annualized_compouded_return =(1 + portfolio_return_series).prod() ** (frequency / portfolio_return_series.count()) - 1
    portfolio_annualized_simple_return = portfolio_return_series.mean()*frequency
    portfolio_annualized_volatility= portfolio_return_series.std()*np.sqrt(frequency)
    portfolio_VaR_05, portfolio_CVaR_05 = VaR_CVaR_Quantile(portfolio_return_series,0.05)
    portfolio_sharp_ratio = portfolio_annualized_compouded_return/portfolio_annualized_volatility
    
    portfolio_kurtosis= scipy.stats.kurtosis(portfolio_return_series)+3
    portfolio_skewness= scipy.stats.skew(portfolio_return_series)
    
    dict_stats={"Annualized Simple Return":[portfolio_annualized_simple_return],
                "Annualized Compunded Return":[portfolio_annualized_compouded_return],
                "Annualized Volatility":[portfolio_annualized_volatility],
                "Sharp Ratio":[portfolio_sharp_ratio],
                "Kurtosis":[portfolio_kurtosis],
                "Skewness":[portfolio_skewness],
                "VaR 5":[portfolio_VaR_05],
                "CVaR 5":[portfolio_CVaR_05]}
    return pd.DataFrame(dict_stats)

In [ ]:
def portfolio_performance_computer(df_return_data,
                                   df_weights,
                                   list_of_assets_columns,
                                   frequency=252):
    """This function utlize portfolio_return_series_computer and portfolio_stats_computer to compute 
    Annualized Simple Return and Annualized Volatility of portfolios. The inputs are:
    df_return_data: return of the assets and 
    df_weights:weights of assets in each portfolio.
    frequency: frequency of the return data"""
    df_portfolios_return_series = portfolio_return_series_computer(df_return_data,
                                                                   df_weights,
                                                                   list_of_assets_columns)
    
    df_portfolios_return_series_stats=df_portfolios_return_series.groupby(
        "portfolio")["return"].apply(lambda x: portfolio_stats_computer(x,frequency=frequency))
    list_of_stats=df_portfolios_return_series_stats.columns.to_list()
    df_portfolios_return_series_stats.reset_index(inplace=True)
    df_portfolios_return_series_stats=pd.concat([df_weights,
                                                 df_portfolios_return_series_stats],
                                                ignore_index=False,axis=1)
    df_portfolios_return_series_stats.sort_values(["Annualized Volatility"],inplace=True)
    df_portfolios_return_series_stats.reset_index(drop=True,inplace=True)
    return df_portfolios_return_series_stats.drop(["portfolio","level_1"],axis=1)

In [ ]:
df_stock_return_data_train.corr()

In [ ]:
def efficient_portfolio_instance(df_return_data,
                                 frequency=252,
                                 covarirance_method="ledoit_wolf",
                                 compound_return=False,
                                 weight_bounds=(0,1)):
    """df_return_data: asset prices or return
    frequency: frequency of your data, by default is 252 (Daily Return)
    compound_return: comound the return or simple return. Set it True for Compounding and False for simple return
    risk_aversion: utility,
    covarirance_method: sample_cov,semicovariance, exp_cov, min_cov_determinant, ledoit_wolf,
    ledoit_wolf_constant_variance, ledoit_wolf_single_factor, ledoit_wolf_constant_correlation,oracle_approximating
    target_risk: amount of annualized volatility. If not specified, it optimzes for the lowest CVaR portfolio"""
    ### Get Annualized Mean and Covairance
    annualized_return = mean_historical_return(df_return_data,
                                               returns_data=True,
                                               frequency=frequency,
                                               compounding=compound_return)
    
    annualized_cov= pypfopt.risk_models.risk_matrix(df_return_data,
                                                    returns_data=True,
                                                    method=covarirance_method,
                                                    frequency=frequency)
    ef= EfficientFrontier(annualized_return,
                          annualized_cov,
                          weight_bounds=weight_bounds)
    return(ef)

In [ ]:
def efficient_frontier_calculator(df_return_data,
                                  frequency=252,
                                  covarirance_method="ledoit_wolf",
                                  compound_return=False,
                                  portfolio_performance=False,
                                  weight_bounds=(0,1),
                                  volatility_point=1000):
    list_of_assets=df_return_data.columns.tolist()
    ef_for_min_vol = efficient_portfolio_instance(df_return_data,
                                                  frequency=frequency,
                                                  covarirance_method=covarirance_method,
                                                  compound_return=compound_return,
                                                  weight_bounds=weight_bounds)
    ef_for_min_vol.min_volatility()
    min_possible_annualized_volatility = ef_for_min_vol.portfolio_performance()[1] + 0.001
    max_possible_annualzied_volatility = (df_return_data.std()*np.sqrt(frequency)).max() + 0.001
    delta = (max_possible_annualzied_volatility - min_possible_annualized_volatility)
    ### Generate enough number of evenly spaced point of volatility
    range_of_annualzied_volatility=np.linspace(min_possible_annualized_volatility,
                                               max_possible_annualzied_volatility,
                                               num=int(delta*volatility_point))
    list_of_df_weights=[]
    for target_risk in range_of_annualzied_volatility:
        try:
            ef = efficient_portfolio_instance(df_return_data,
                                             frequency=frequency,
                                             covarirance_method=covarirance_method,
                                             compound_return=compound_return,
                                             weight_bounds=weight_bounds)
            ef.efficient_risk(target_risk)
            ### Get The Weights of asset in portfolio
            df_weights = pd.DataFrame([ef.weights],
                                      columns=list_of_assets)
            df_weights["target_risk"]=target_risk
            list_of_df_weights.append(df_weights)
        except:
            pass
    
    ### Concatinate all the Portfolios with target risk
    df_efficient_frontier=pd.concat(list_of_df_weights,
                                    ignore_index=True)
    
    ### There are replicated portfolios, get rid of them
    df_efficient_frontier=df_efficient_frontier.drop_duplicates(list_of_assets).reset_index(drop=True)
    ### Computer Portfolio Performance
    if portfolio_performance:
        df_efficient_frontier=portfolio_performance_computer(df_return_data,
                                                             df_efficient_frontier,
                                                             list_of_assets,
                                                             frequency=frequency)
    return(df_efficient_frontier)

In [ ]:
def data_shaker(df_return_data,
                shake_magnitude=.05,
                multivariate_normal=True):
    """Add some random white noise with mean zeror some magnitude of the covariance matrix
    multivariate_normal: Add some random white noise with mean zeror magnitude of the standard deviation of the underlying asset. """
    if multivariate_normal:
        random_noise= np.random.multivariate_normal(df_return_data.mean()*0,
                                                    df_return_data.cov(),
                                                    len(df_return_data))
        df_return_data_with_random_noise= df_return_data + (random_noise*shake_magnitude)
    
    if multivariate_normal == False:
        number_of_observation=df_return_data.shape[0]
        number_of_asset=df_return_data.shape[1]
        data_std=df_return_data.std()
        random_noise= np.random.normal(loc=0,
                                      scale=data_std,
                                      size=(number_of_observation,number_of_asset))
        df_return_data_with_random_noise= df_return_data + (random_noise*shake_magnitude)
    return(df_return_data_with_random_noise)

In [ ]:
def multivariate_normal_return_simulator(df_return_data,
                                         number_of_return_series_to_be_simulated):
    """multivariate_normal Return simulator"""
    list_of_assets = df_return_data.columns.to_list()
    ### Simulate the returns
    simulated_returns = np.random.multivariate_normal(df_return_data.mean(),
                                                      df_return_data.cov(),
                                                      number_of_return_series_to_be_simulated)
    ### Put those simulated returns into DataFrame
    df_simulated_returns = pd.DataFrame(simulated_returns,
                                        columns=list_of_assets)
    return(df_simulated_returns)

In [ ]:
def efficient_frontier_simulator_multivariate_normal(df_return_data,
                                                    number_of_simulation=10,
                                                    frequency=252,
                                                    compound_return=False,
                                                    covarirance_method="ledoit_wolf",
                                                    weight_bounds=(0, 1),
                                                    volatility_point=1000):
    """ Simulate Multi variate Normal Distribution return from actual data. Then construct number_of_simulation efficinet 
    frontier.
    df_return_data: returns of assets
    number_of_simulation: number of simulation to run
    frequency: frequency of data, daily=252, monthly =12 etc.
    compound_return: compouding return or simple return"""
    list_of_assets=df_return_data.columns.tolist()
    ### Consurtct Efficient Frontiter with simulated Return and Covriance matrix
    list_of_simulated_portfolios=[]
    for i in range(number_of_simulation):
        df_data_return_simulated= multivariate_normal_return_simulator(df_return_data,
                                                                       len(df_return_data))
        ### Don't calcualte portfolio perfomrnace the with efficient_frontier_calculator
        ### Because the returns are simulated return, not actual
        df_weights = efficient_frontier_calculator(df_data_return_simulated,
                                                   frequency=frequency,
                                                   compound_return=compound_return,
                                                   covarirance_method=covarirance_method,
                                                   portfolio_performance=False,
                                                   weight_bounds=weight_bounds,
                                                   volatility_point=volatility_point)
        df_weights["Simulation"]=i
        list_of_simulated_portfolios.append(df_weights)
    ### Concatinate all the simulated returns into 1 dataframe
    df_simulated_efficient_frontier = pd.concat(list_of_simulated_portfolios,
                                                ignore_index=True)
    
    ### Compute the return and volatility of each portfolio based on actual return data
    df_simulated_efficient_frontier= portfolio_performance_computer(df_return_data,
                                                                    df_simulated_efficient_frontier,
                                                                    list_of_assets,
                                                                    frequency)
    
    return(df_simulated_efficient_frontier)

In [ ]:
def efficient_frontier_simulator_shaking_returns(df_return_data,
                                                 list_of_shaking_magnitude=[10,15,20,25,30,35,40],
                                                 frequency=252,
                                                 compound_return=False,
                                                 covarirance_method="ledoit_wolf",
                                                 multivariate_normal=True,
                                                 volatility_point=1000,
                                                 weight_bounds=(0,1)):
    list_of_assets=df_return_data.columns.tolist()
    """ Inject white noise with mean 0 and std*%shaking_magnitude into the resturn to consturct 
    multiple efficient frontier
    df_return_data: Retuns of asstes
    list_of_shaking_magnitude: list of magnitude to shake the returns, based on percentage of standard deviation of the
    underlying asset  20% 10% etc.
    frequency: data frequency. Default 252 daily. 12 monthly etc.
    """
    list_of_simulated_portfolios=[]
    for shake in list_of_shaking_magnitude:
        shake=shake/100
        df_data_return_simulated = data_shaker(df_return_data,
                                               shake_magnitude=shake,
                                               multivariate_normal=multivariate_normal)
        ### Don't calcualte portfolio perfomrnace the with efficient_frontier_calculator
        ### Because the returns are simulated return, not actual
        df_weights = efficient_frontier_calculator(df_data_return_simulated,
                                                   frequency=frequency,
                                                   compound_return=compound_return,
                                                   covarirance_method=covarirance_method,
                                                   portfolio_performance=False,
                                                   weight_bounds=weight_bounds,
                                                   volatility_point=volatility_point)
        df_weights["shake"] = shake
        list_of_simulated_portfolios.append(df_weights)
    
    ### Concatinate all the portfolios
    df_simulated_efficient_frontier=pd.concat(list_of_simulated_portfolios,
                                              ignore_index=True)
    ### Compute Portfolios performances
    df_simulated_efficient_frontier=portfolio_performance_computer(df_return_data,
                                                                   df_simulated_efficient_frontier,
                                                                   list_of_assets,
                                                                   frequency)
    return (df_simulated_efficient_frontier)

In [ ]:
def data_bin_qty_comuter(values,
                         spacing):
    import pandas as pd
    import numpy as np
    """Determine how many bins are needed for eqaully spaced bins with the spacing specified"""
    values_series=pd.Series(values)
    values_series_min=values_series.min()
    values_series_max=values_series.max()
    values_series_evenly_spaced=np.arange(values_series_min-spacing,
                                          values_series_max+spacing,
                                          spacing)
    number_of_bins_for_splititng_volatilies= len(values_series_evenly_spaced)
    return(number_of_bins_for_splititng_volatilies)

def portfolio_resampler(volatility_series,
                        volatility_spacing):
    volatility_series=pd.Series(volatility_series)
    number_of_bins = data_bin_qty_comuter(values = volatility_series,
                                          spacing =volatility_spacing)
    return (pd.cut(volatility_series,number_of_bins))

def efficient_frontier_resampler(df_efficient_frontiers,
                                 df_return_data,
                                 volatility_spacing=.005,
                                 frequency=252):
    list_of_assets = df_return_data.columns.tolist()
    df_weights_resampled = df_efficient_frontiers.copy()
    ### Resmaple the portfolios based on volatility bins
    df_weights_resampled["Volatility Bins"]= portfolio_resampler(df_weights_resampled["Annualized Volatility"],
                                                                 volatility_spacing)
    df_weights_resampled["Volatility Bins"]=df_weights_resampled["Volatility Bins"].astype(str)
    ### Get the Average weight of each asset in resampled portfolios from each bin
    df_weights_resampled = df_weights_resampled.groupby(["Volatility Bins"])[list_of_assets].mean().reset_index().dropna()
    ### Compute the performance of the resmapled portfolios
    df_weights_resampled=portfolio_performance_computer(df_return_data,
                                                        df_weights_resampled,
                                                        list_of_assets,
                                                        frequency)
    return(df_weights_resampled)

In [ ]:
def efficient_portfolio_CVaR_calculator(df_return_data,
                                        CVaR_Significance=0.05,
                                        target_risk=None,
                                        frequency=252,
                                        compound_return=False,
                                        weight_bounds=(0,1),
                                        covarirance_method="sample_cov",
                                        solver="SLSQP",
                                        portfolio_performance=False):
    """covarirance_method: sample_cov,semicovariance, exp_cov, min_cov_determinant, ledoit_wolf,
    ledoit_wolf_constant_variance, ledoit_wolf_single_factor, ledoit_wolf_constant_correlation,oracle_approximating
    target_risk: amount of annualized volatility. If not specified, it optimzes for the lowest CVaR portfolio"""
    list_of_assets= df_return_data.columns.tolist()
    ### CVAR Faunction to be minimiazedd
    def CVaR_Quantile_Method(w,
                             return_series,
                             significance_level):
        import scipy.stats
        import numpy as np
        data_series = return_series.dot(w)
        mean = np.mean(data_series)
        std = np.std(data_series)
        ### Data Driven VaR CVaR
        VaR_Quantile = np.quantile(data_series,significance_level)
        return_in_tail_based_on_VaR_Quantile=data_series[data_series<VaR_Quantile]
        CVaR_Quantile=(np.mean(return_in_tail_based_on_VaR_Quantile))*(-1)
        return(CVaR_Quantile)
    
    ef = efficient_portfolio_instance(df_return_data,
                                      frequency=frequency,
                                      covarirance_method=covarirance_method,
                                      compound_return=compound_return,
                                      weight_bounds=weight_bounds)
    
    ### Minimize the CVaR function subject to no constrian
    if target_risk==None:
        ef.nonconvex_objective(CVaR_Quantile_Method,
                               solver=solver,
                               weights_sum_to_one=True,
                               objective_args=(df_return_data.values,
                                               CVaR_Significance))
    else:
        # Portfolio vol less than target vol
        risk_constr = {"type": "eq",
                       "fun": lambda w: target_risk ** 2 - np.dot(w.T, np.dot(ef.cov_matrix, w))}
        constraints = [risk_constr]
        ### Minimize the CVaR function with subject to Volatility constrian
        ef.nonconvex_objective(CVaR_Quantile_Method,
                               solver=solver,
                               constraints=constraints,
                               weights_sum_to_one=True,
                               objective_args=(df_return_data.values,
                                               CVaR_Significance))    
    ### Get The Weights of asset in portfolio
    #### Store the outputs into the dataframe 
    df_weights =  pd.DataFrame([ef.weights],
                                      columns=list_of_assets)
    df_weights["CVaR Significance"]=CVaR_Significance
    
    ### Computer Portfolio Performance
    if portfolio_performance:
        df_weights=portfolio_performance_computer(df_return_data,
                                                  df_weights,
                                                  list_of_assets,
                                                  frequency=frequency)
    return(df_weights)

In [ ]:
def efficient_frontier_CVaR_calculator(df_return_data,
                                       CVaR_Significance=0.05,
                                       frequency=252,
                                       compound_return=False,
                                       weight_bounds=(0,1),
                                       covarirance_method="sample_cov",
                                       solver="SLSQP",
                                       portfolio_performance=False,
                                       volatility_point=1000):
    """covarirance_method: sample_cov,semicovariance, exp_cov, min_cov_determinant, ledoit_wolf,
    ledoit_wolf_constant_variance, ledoit_wolf_single_factor, ledoit_wolf_constant_correlation,oracle_approximating
    volatility_points: Number of evenly spaced volatility portfolio to be created if you need more precise volatility CVaR portfolio,
    increase it. Will be computationaly more expensive """
    list_of_assets= df_return_data.columns.tolist()
    ### CVAR Faunction to be minimiazedd
    def CVaR_Quantile_Method(w,
                             return_series,
                             significance_level):
        import scipy.stats
        import numpy as np
        data_series = return_series.dot(w)
        mean = np.mean(data_series)
        std = np.std(data_series)
        ### Data Driven VaR CVaR
        VaR_Quantile = np.quantile(data_series,significance_level)
        return_in_tail_based_on_VaR_Quantile=data_series[data_series<VaR_Quantile]
        CVaR_Quantile=(np.mean(return_in_tail_based_on_VaR_Quantile))*(-1)
        return(CVaR_Quantile)

    
    ef = efficient_portfolio_instance(df_return_data,
                                      frequency=frequency,
                                      covarirance_method=covarirance_method,
                                      compound_return=compound_return,
                                      weight_bounds=weight_bounds)
    
    ### Compute Min Volatility Portfolio
    ef.min_volatility()
    min_possible_annualized_volatility = ef.portfolio_performance()[1] + 0.001
    max_possible_annualzied_volatility = (df_return_data.std()*np.sqrt(frequency)).max() +  0.001
    delta = (max_possible_annualzied_volatility - min_possible_annualized_volatility)
    
    
    ### Generate enough number of evenly spaced point of volatility
    range_of_annualzied_volatility=np.linspace(min_possible_annualized_volatility,
                                               max_possible_annualzied_volatility,
                                               num=int(delta*volatility_point))
    
    ef = efficient_portfolio_instance(df_return_data,
                                      frequency=frequency,
                                      covarirance_method=covarirance_method,
                                      compound_return=compound_return,
                                      weight_bounds=weight_bounds)
    
    #### Portfolio vol less than target vol
    risk_constr = {"type": "eq",
                   "fun": lambda w: target_risk ** 2 - np.dot(w.T, np.dot(ef.cov_matrix, w))}
    constraints = [risk_constr]
    list_of_df_weights=[]
    ### Minimize the CVaR function with subject to Volatility constrian
    for target_risk in range_of_annualzied_volatility:
        try:
            ef.nonconvex_objective(CVaR_Quantile_Method,
                                   solver=solver,
                                   weights_sum_to_one=True,
                                   constraints=constraints,
                                   objective_args=(df_return_data.values,
                                                   CVaR_Significance))
            ### Get The Weights of asset in portfolio
            df_weights =  pd.DataFrame([ef.weights],
                                      columns=list_of_assets)
            #### Store the outputs into the dataframe 
            df_weights["CVaR Significance"]=CVaR_Significance
            list_of_df_weights.append(df_weights)
        except:
            pass
        
    ### Concatinate all the CVaR Portfolios
    df_efficient_frontier=pd.concat(list_of_df_weights,
                                    ignore_index=True)
  
    ### Computer Portfolio Performance
    if portfolio_performance:
        df_efficient_frontier=portfolio_performance_computer(df_return_data,
                                                             df_efficient_frontier,
                                                             list_of_assets,
                                                             frequency=frequency)
    return(df_efficient_frontier)

In [ ]:
def efficient_frontier_CVaR_simulator_shaking_returns(df_return_data,
                                                     list_of_shaking_magnitude=[30,35,40,45,50],
                                                      multivariate_normal=True,
                                                     CVaR_Significance=0.05,
                                                     frequency=252,
                                                     compound_return=False,
                                                     weight_bounds=(0,1),
                                                     covarirance_method="sample_cov",
                                                     solver="SLSQP",
                                                     volatility_point=1000):
    list_of_assets= df_return_data.columns.tolist()
    list_of_simulated_portfolios=[]
    #### Shake the retruns with some percentage of their standadrd deviation- Adding white noise with mean zero.
    for shake in list_of_shaking_magnitude:
        shake=shake/100
        df_return_data_shaked = data_shaker(df_return_data=df_return_data,
                                            shake_magnitude=shake,
                                            multivariate_normal=multivariate_normal)
        
        df_weights= efficient_frontier_CVaR_calculator(df_return_data=df_return_data_shaked,
                                                      CVaR_Significance=CVaR_Significance,
                                                      frequency=frequency,
                                                      compound_return=compound_return,
                                                      weight_bounds=weight_bounds,
                                                      covarirance_method=covarirance_method,
                                                      solver=solver,
                                                      portfolio_performance=False,
                                                      volatility_point=volatility_point)
        df_weights["shake"]=shake
        list_of_simulated_portfolios.append(df_weights)
    ### Concatinate all the portfolios
    df_simulated_efficient_frontier = pd.concat(list_of_simulated_portfolios,
                                                ignore_index=True)
    
    ### Compute Portfolios performances
    df_simulated_efficient_frontier = portfolio_performance_computer(df_return_data,
                                                         df_simulated_efficient_frontier,
                                                         list_of_assets,
                                                         frequency)
    return (df_simulated_efficient_frontier)

### Construct Non-Resmapled Efficient Frontier

In [ ]:
list_of_all_assets= df_stock_return_data.columns.to_list()

In [ ]:
%%time
df_weights_MVO=efficient_frontier_calculator(df_stock_return_data,
                                  frequency=252,
                                  covarirance_method="ledoit_wolf",
                                  compound_return=False,
                                  portfolio_performance=True,
                                  weight_bounds=(0,1),
                                  volatility_point=1800)

In [ ]:
### Plot Efficient Frontier
plt.scatter(df_weights_MVO["Annualized Volatility"],
            df_weights_MVO["Annualized Simple Return"])
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Simple Return")
plt.title("MVO Efficient Frontier")
plt.show()

In [ ]:
df_weights_MVO.sort_values("Sharp Ratio",
                           ascending=False)

In [ ]:
### Plot Asset Allocation
df_weights_MVO.set_index("Annualized Volatility")[list_of_all_assets].plot.area(figsize=(10,5))
plt.legend()
plt.xlabel("Portfolio")
plt.ylabel("Weight")
plt.title("MVO Portfolio Asset Allocations")
plt.show()

### Construct Resmapled Efficient Frontier (multi Variate Normal)
Consurtct Efficient Frontiter with simulated Return and Covriance matrix for 50 times and then take the average weights of each asset for earh risk averstion 

check out emperical multivariete distribution:
1. https://machinelearningmastery.com/empirical-distribution-function-in-python/
2. https://stats.stackexchange.com/questions/62146/method-for-generating-correlated-non-normal-data
3. http://www.econometricsbysimulation.com/2014/02/easily-generate-correlated-variables.html
4. https://www.newfrontieradvisors.com/media/1138/estimation-error-and-portfolio-optimization-12-05.pdf

In [ ]:
%%time
#### Simulate N number of efficient Frontier via Multivarite Normal Distribution
df_weights_Multi_Normal_MVO_simulated = efficient_frontier_simulator_multivariate_normal(df_stock_return_data,
                                                                                number_of_simulation=30,
                                                                                frequency=252,
                                                                                compound_return=False,
                                                                                covarirance_method="ledoit_wolf",
                                                                                weight_bounds=(0, 1),
                                                                                volatility_point=1000)

In [ ]:
### resample the efficient frontier from N efficient frontier computed above
df_weights_Multi_Normal_RMVO= efficient_frontier_resampler(df_weights_Multi_Normal_MVO_simulated,
                                                           df_stock_return_data,
                                                           volatility_spacing=0.02,
                                                           frequency=252)

In [ ]:
### Plot Efficient Frontier
df_weights_Multi_Normal_MVO_simulated.groupby("Simulation").apply(lambda x: plt.plot(x["Annualized Volatility"],
                                                                                     x["Annualized Simple Return"],
                                                                                    color="lightblue"))
plt.plot(df_weights_Multi_Normal_RMVO["Annualized Volatility"],
         df_weights_Multi_Normal_RMVO["Annualized Simple Return"],
        color="red")
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Simple Return")
plt.title("Resampled Efficient Frontier")
plt.show()

In [ ]:
### Plot Asset Allocation
df_weights_Multi_Normal_RMVO.set_index("Annualized Volatility")[list_of_all_assets].plot.area(figsize=(8.5,4.5))
plt.xlabel("Annualized Volatility")
plt.ylabel("Weight")
plt.title("Portfolio Allocations via Resampling (Multi Variate Return) via Risk Aversion")
plt.show()

### Construct Resmapled Efficient Frontier (Shaking)
Consurtct Efficient Frontiter with shaking Return and Covriance matrix. 
Then take the average weights of portfolios in volatility buckets 

In [ ]:
%%time
df_weights_shaked_MVO_simulated_MN1 = efficient_frontier_simulator_shaking_returns(df_stock_return_data,
                                                                               list_of_shaking_magnitude=np.arange(25,35,.5),
                                                                               multivariate_normal=True,
                                                                               frequency=252,
                                                                               compound_return=False,
                                                                               covarirance_method="sample_cov",
                                                                               volatility_point=1800,
                                                                               weight_bounds=(0,1))

In [ ]:
df_weights_shaked_MVO_simulated_MN1_resampled=efficient_frontier_resampler(df_weights_shaked_MVO_simulated_MN1,
                                                     df_stock_return_data,
                                                     volatility_spacing=0.02,
                                                     frequency=252)

df_weights_shaked_MVO_simulated_MN1.groupby(['shake']).apply(lambda x: plt.plot(x["Annualized Volatility"],
                                                                                x["Annualized Simple Return"],
                                                                                color="lightblue"))

plt.plot(df_weights_shaked_MVO_simulated_MN1_resampled["Annualized Volatility"],
         df_weights_shaked_MVO_simulated_MN1_resampled["Annualized Simple Return"],
         color='red')
plt.show()

### Plot Asset Allocation
df_weights_shaked_MVO_simulated_MN1_resampled.set_index("Annualized Volatility")[list_of_all_assets].plot.area(figsize=(10,5))
plt.xlabel("Annualized Volatility")
plt.ylabel("Weight")
plt.title("Portfolio Allocations via Resampling - Shaking Returns Series")
plt.show()

### CVaR

In [ ]:
efficient_portfolio_CVaR_calculator(df_stock_return_data,
                                        CVaR_Significance=0.05,
                                        target_risk=None,
                                        frequency=252,
                                        compound_return=False,
                                        weight_bounds=(0,1),
                                        covarirance_method="ledoit_wolf",
                                        solver="SLSQP",
                                        portfolio_performance=True)

In [ ]:
%%time
df_weights_cvar_MVO = efficient_frontier_CVaR_calculator(df_stock_return_data,
                                                         CVaR_Significance=0.05,
                                                         frequency=252,
                                                         compound_return=False,
                                                         weight_bounds=(0,1),
                                                         covarirance_method="sample_cov",
                                                         solver="SLSQP",
                                                         portfolio_performance=True,
                                                         volatility_point=1500)

In [ ]:
### Plot Efficient Frontier
plt.plot(df_weights_cvar_MVO["Annualized Volatility"],
         df_weights_cvar_MVO["Annualized Simple Return"])
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Simple Return")
plt.title("CVaR Efficient Portfolios")
plt.show()

In [ ]:
df_weights_cvar_MVO

In [ ]:
efficient_frontier_resampler(df_weights_cvar_MVO,
                             df_stock_return_data,
                             volatility_spacing=0.004,
                             frequency=252)[list_of_all_assets].plot.area()

In [ ]:
### Plot CVaR Asset Allocation
df_weights_cvar_MVO[list_of_all_assets].plot.area(figsize=(7,5))
plt.xlabel("Portfolio")
plt.ylabel("Weight")
plt.title("CVaR Asset Allocation")
plt.show()

### Resmapled CVaR

In [ ]:
%%time
df_weights_shaked_cvar_mvo = efficient_frontier_CVaR_simulator_shaking_returns(df_stock_return_data,
                                                     list_of_shaking_magnitude=np.arange(25,35),
                                                      multivariate_normal=True,
                                                     CVaR_Significance=0.05,
                                                     frequency=252,
                                                     compound_return=False,
                                                     weight_bounds=(0,1),
                                                     covarirance_method="sample_cov",
                                                     solver="SLSQP",
                                                     volatility_point=1500)

In [ ]:
### resample the efficient frontier from N efficient frontier computed above
df_weights_cvar_resampled_mvo= efficient_frontier_resampler(df_weights_shaked_cvar_mvo,
                                                            df_stock_return_data,
                                                            volatility_spacing=0.005,
                                                            frequency=252)

In [ ]:
df_weights_shaked_cvar_mvo.groupby("shake").apply(lambda x: plt.plot(x["Annualized Volatility"],
                                                                     x["Annualized Simple Return"],
                                                                    color="lightblue"))
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Simple Return")
### Plot Efficient Frontier
plt.plot(df_weights_cvar_resampled_mvo["Annualized Volatility"],
            df_weights_cvar_resampled_mvo["Annualized Simple Return"],color="red")
plt.xlabel("Annualized Volatility")
plt.ylabel("Annualized Simple Return")
plt.title("Rasampled CVaR Efficient Portfolios")
plt.show()

In [ ]:
### Plot CVaR Asset Allocation
df_weights_cvar_resampled_mvo.set_index("Annualized Volatility")[df_stock_return_data.columns].plot.area(figsize=(10,4.5))
plt.xlabel("Annualized Volatility")
plt.ylabel("Weight")
plt.title("Resampled Portfolio Allocation (CVaR)")
plt.show()